<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/sdxl_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Install** { display-mode: "form" }
%%capture
!pip install -q invisible_watermark transformers accelerate safetensors diffusers controlnet_aux==0.0.7 xformers mediapy
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline, DiffusionPipeline, KDPM2AncestralDiscreteScheduler, StableDiffusionXLPipeline, AutoencoderKL
import gc
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display
import os
import random
model_base = "stabilityai/stable-diffusion-xl-base-1.0"
v_autoencoder = "madebyollin/sdxl-vae-fp16-fix" # fix vae for run in fp16 precision without generating NaNs

vae = AutoencoderKL.from_pretrained(v_autoencoder, torch_dtype=torch.float16)

pipe = StableDiffusionXLPipeline.from_pretrained(
    model_base,
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    add_watermarker=False, # no watermarker
    )

pipe.safety_checker = None

pipe.to("cuda")
model_refiner = "stabilityai/stable-diffusion-xl-refiner-1.0"

pipe_refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    model_refiner,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    add_watermarker=False, # no watermarker
    )

#pipe_refiner.to("cuda")
pipe_refiner.enable_model_cpu_offload()
#(Optional) Change the scheduler
pipe.scheduler = KDPM2AncestralDiscreteScheduler.from_config(
pipe.scheduler.config, use_karras_sigmas=False
)
#generator = torch.Generator().manual_seed(42)

import os
import torch
import random
import gc
from diffusers import StableDiffusionXLPipeline
from diffusers import DPMSolverMultistepScheduler, EulerDiscreteScheduler
from diffusers import KDPM2DiscreteScheduler, KDPM2AncestralDiscreteScheduler
from diffusers import EulerAncestralDiscreteScheduler, HeunDiscreteScheduler
from diffusers import LMSDiscreteScheduler

model_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = StableDiffusionXLPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

save_dir = './samples_sdxl'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

def set_sampler(pipe, sampler_name):
    if sampler_name == "euler":
        pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
    return pipe

def set_scheduler(pipe, scheduler_name, num_steps):
    if scheduler_name == "karras":
        pipe.scheduler.use_karras_sigmas = True
    elif scheduler_name == "exponential":
        pipe.scheduler.set_timesteps(num_steps, decay_rate=0.5)
    return pipe


In [ ]:
#@title ##**Create** { display-mode: "form" }
prompt =  "cyberpunk, cat, Ukrainian flag" #@param {type:"string"}
prompt2 = "" #@param {type:"string"}
negative_prompt = '' #@param {type:"string"}
negative_prompt2 = '' #@param {type:"string"}
scale = 9 #@param {type:"integer"}
steps = 50 #@param {type:"integer"}
height = 1024 #@param {type:"integer"}
width = 1024 #@param {type:"integer"}
sampler_name = "LMS" #@param ['euler', 'LMS', 'ddim', 'heun', "euler_cfg_pp", "euler_ancestral", "euler_ancestral_cfg_pp", "heunpp2", "dpm_2", "dpm_2_ancestral", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "dpmpp_3m_sde", "dpmpp_3m_sde_gpu", "ddpm", "Icm", "ipndm", "ipndm_v", "deis", "uni_pc", "uni_pc_bh2"]
scheduler_name = "karras" #@param ['simple', 'karras', 'normal', 'exponential', 'sgm_uniform' , 'ddim_uniform' , 'beta']

params = {
    "prompt": prompt,
    "prompt_2": prompt2,
    "height": height,
    "width": width,
    "negative_prompt": negative_prompt,
    "negative_prompt_2": negative_prompt2,
    "guidance_scale": scale,
    "num_inference_steps": steps
}

seed = random.randint(0, 1000000000000000000)
generator = torch.Generator(device='cuda').manual_seed(seed)

pipe = set_sampler(pipe, sampler_name)
pipe = set_scheduler(pipe, scheduler_name, steps)

sdxl_img = pipe(**params, generator=generator).images[0]

sdxl_img.save(os.path.join(save_dir, f"seed_{seed}_steps_{steps}_{sampler_name}_{scheduler_name}.png"))

gc.collect()
torch.cuda.empty_cache()

print(f"Изображение создано и сохранено с параметрами: sampler={sampler_name}, scheduler={scheduler_name}, seed={seed}")

In [ ]:
#@title ##**Show** { display-mode: "form" }
sdxl_img